In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')

In [ ]:
print('Training set:', train.shape)
print('Test set: ', test.shape)


In [ ]:
X_train = train
X_test = test
Y_train = X_train.pop('label')
X_test.pop('id')

In [ ]:
'''Preprocess'''
import numpy as np
X_train, X_test = np.array(X_train)/255.0, np.array(X_test)/255.0


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28,28,1)

In [ ]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train, Y_train)

In [ ]:
#Create validation set
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)
print('Training set:', X_train.shape, Y_train.shape)
print('Validation set:', X_valid.shape, Y_valid.shape)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, LeakyReLU

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(filters=32, kernel_size=[3,3],input_shape=[28,28,1], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=[3,3], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=[3,3], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))
model.add(Conv2D(filters=64, kernel_size=[3,3], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256, kernel_size=[3,3], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))
model.add(Conv2D(filters=256, kernel_size=[3,3], padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(0.05))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator (rotation_range = 10,
width_shift_range = 0.25,
height_shift_range = 0.25,
shear_range = 0.1,
zoom_range = 0.25,
horizontal_flip = False)

In [ ]:
batch_size =  32
model.fit(datagen.flow(X_train, Y_train), batch_size=batch_size, epochs=20, validation_data=(X_valid, Y_valid))

In [ ]:
test.head(5)

In [ ]:
test = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
id = test['id']

In [ ]:
predict = model.predict(X_test)

In [ ]:
predict = np.argmax(predict, axis=1)

In [ ]:
predict.shape

In [ ]:
submission = pd.DataFrame({'id':id,'label':predict})

In [ ]:
filename = 'submission.csv'
submission.to_csv(filename, index=False)

In [ ]:
submission.head(5)